### レースデータを取得し登録するコード

基本的に完全に動いている！！

In [1]:
import sys
import os
import logging
from datetime import datetime, date
import pandas as pd
import json

# プロジェクトのルートディレクトリをパスに追加
sys.path.append('..')

# ログ設定
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("=== 競馬レースデータ収集システム ===")
print(f"実行時刻: {datetime.now()}")


=== 競馬レースデータ収集システム ===
実行時刻: 2025-08-31 21:56:00.102694


In [2]:
# 必要なモジュールをインポート
from src.scraping.scrapers.race_scraper import RaceScraper
from src.scraping.storage.race_storage import RaceStorage
from src.database.schemas.race_schema import Race, RaceResult
from src.scraping.storage.postgresql_storage import PostgreSQLStorage

# スクレイパーとストレージを初期化
scraper = RaceScraper(delay=1.0)  # 1秒間隔でリクエスト
storage = RaceStorage()

print("✅ システム初期化完了")

✅ システム初期化完了


In [5]:
# レースリストを取得
print("=== Testing Race List Extraction ===")
race_list = scraper.scrape_race_list_by_conditions(
    start_year=2010,
    end_year=2018,
    grades=['1'],
    limit=200
)

print(f"Found {len(race_list)} races")
for race in race_list[:3]:  # 最初の3件のみ表示
    print(f"  - {race}")

2025-08-31 21:57:29,402 - src.scraping.scrapers.race_scraper - INFO - Fetching race list: 2010-2018, grades: ['1']


=== Testing Race List Extraction ===


2025-08-31 21:57:31,500 - src.scraping.scrapers.race_scraper - INFO - Final URL: https://db.netkeiba.com/?pid=race_list&word=&start_year=2010&start_mon=none&end_year=2018&end_mon=none&list=200&sort=date&track%5B%5D=1&jyo%5B%5D=01&jyo%5B%5D=02&jyo%5B%5D=03&jyo%5B%5D=04&jyo%5B%5D=05&jyo%5B%5D=06&jyo%5B%5D=07&jyo%5B%5D=08&jyo%5B%5D=09&jyo%5B%5D=10&grade%5B%5D=1
2025-08-31 21:57:31,501 - src.scraping.scrapers.race_scraper - INFO - Response status: 200
2025-08-31 21:57:31,530 - src.scraping.extractors.race.race_list_extractor - INFO - Extracted 20 races from list
2025-08-31 21:57:31,531 - src.scraping.scrapers.race_scraper - INFO - Found 20 races


Found 20 races
  - {'race_name': 'ホープフルS(GI)', 'race_id': '201806050911'}
  - {'race_name': '有馬記念(GI)', 'race_id': '201806050811'}
  - {'race_name': '朝日フューチュリティ(GI)', 'race_id': '201809050611'}


In [4]:
# レース詳細と結果を取得して保存
storage = PostgreSQLStorage()

# race_list = race_list[:3]  # テストのため3件に制限

for race in race_list:
    race_detail, race_results = scraper.scrape_race_detail(race['race_id'])

    print(f"現在処理中のレース: {race_detail}")

    storage.insert_race(race_detail)
    storage.insert_race_results(race_results)

2025-08-31 21:56:28,194 - src.scraping.storage.postgresql_storage - INFO - PostgreSQL connection test successful
2025-08-31 21:56:28,195 - src.scraping.scrapers.race_scraper - INFO - race detail url: https://db.netkeiba.com/race/201806050911/
2025-08-31 21:56:28,584 - src.scraping.extractors.race.race_detail_extractor - INFO - Extracted 13 race results for 201806050911
2025-08-31 21:56:28,584 - src.scraping.scrapers.race_scraper - INFO - Successfully extracted race data: 201806050911 (13 horses)
2025-08-31 21:56:28,585 - src.scraping.storage.postgresql_storage - INFO - Race Is: Race(race_id='201806050911', race_date=datetime.date(2018, 12, 28), track_name='中山', race_number=11, race_name='第35回ホープフルステークス(GI)', distance=2000, track_type='芝', total_horses=13, grade='G1', track_direction='右', weather='晴', track_condition='良', start_time=datetime.time(15, 30), winning_time='2:01.6', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)
2025-08-31

現在処理中のレース: Race(race_id='201806050911', race_date=datetime.date(2018, 12, 28), track_name='中山', race_number=11, race_name='第35回ホープフルステークス(GI)', distance=2000, track_type='芝', total_horses=13, grade='G1', track_direction='右', weather='晴', track_condition='良', start_time=datetime.time(15, 30), winning_time='2:01.6', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)


2025-08-31 21:56:29,222 - src.scraping.extractors.race.race_detail_extractor - INFO - Extracted 16 race results for 201806050811
2025-08-31 21:56:29,223 - src.scraping.scrapers.race_scraper - INFO - Successfully extracted race data: 201806050811 (16 horses)
2025-08-31 21:56:29,223 - src.scraping.storage.postgresql_storage - INFO - Race Is: Race(race_id='201806050811', race_date=datetime.date(2018, 12, 23), track_name='中山', race_number=11, race_name='第63回有馬記念(GI)', distance=2500, track_type='芝', total_horses=16, grade='G1', track_direction='右', weather='曇', track_condition='稍重', start_time=datetime.time(15, 25), winning_time='2:32.2', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)
2025-08-31 21:56:29,379 - src.scraping.scrapers.race_scraper - INFO - race detail url: https://db.netkeiba.com/race/201809050611/


現在処理中のレース: Race(race_id='201806050811', race_date=datetime.date(2018, 12, 23), track_name='中山', race_number=11, race_name='第63回有馬記念(GI)', distance=2500, track_type='芝', total_horses=16, grade='G1', track_direction='右', weather='曇', track_condition='稍重', start_time=datetime.time(15, 25), winning_time='2:32.2', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)


2025-08-31 21:56:30,429 - src.scraping.extractors.race.race_detail_extractor - INFO - Extracted 15 race results for 201809050611
2025-08-31 21:56:30,430 - src.scraping.scrapers.race_scraper - INFO - Successfully extracted race data: 201809050611 (15 horses)
2025-08-31 21:56:30,430 - src.scraping.storage.postgresql_storage - INFO - Race Is: Race(race_id='201809050611', race_date=datetime.date(2018, 12, 16), track_name='阪神', race_number=11, race_name='第70回朝日フューチュリティ(GI)', distance=1600, track_type='芝', total_horses=15, grade='G1', track_direction='右', weather='小雨', track_condition='良', start_time=datetime.time(15, 40), winning_time='1:33.9', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)


...

現在処理中のレース: Race(race_id='201808030411', race_date=datetime.date(2018, 4, 29), track_name='京都', race_number=11, race_name='第157回天皇賞(春)(GI)', distance=3200, track_type='芝', total_horses=17, grade='G1', track_direction='右', weather='晴', track_condition='良', start_time=datetime.time(15, 40), winning_time='3:16.2', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)


2025-08-31 21:56:44,066 - src.scraping.extractors.race.race_detail_extractor - INFO - Extracted 16 race results for 201806030811
2025-08-31 21:56:44,067 - src.scraping.scrapers.race_scraper - INFO - Successfully extracted race data: 201806030811 (16 horses)
2025-08-31 21:56:44,067 - src.scraping.storage.postgresql_storage - INFO - Race Is: Race(race_id='201806030811', race_date=datetime.date(2018, 4, 15), track_name='中山', race_number=11, race_name='第78回皐月賞(GI)', distance=2000, track_type='芝', total_horses=16, grade='G1', track_direction='右', weather='曇', track_condition='稍重', start_time=datetime.time(15, 40), winning_time='2:00.8', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)
2025-08-31 21:56:44,211 - src.scraping.scrapers.race_scraper - INFO - race detail url: https://db.netkeiba.com/race/201809020611/


現在処理中のレース: Race(race_id='201806030811', race_date=datetime.date(2018, 4, 15), track_name='中山', race_number=11, race_name='第78回皐月賞(GI)', distance=2000, track_type='芝', total_horses=16, grade='G1', track_direction='右', weather='曇', track_condition='稍重', start_time=datetime.time(15, 40), winning_time='2:00.8', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)


2025-08-31 21:56:44,828 - src.scraping.extractors.race.race_detail_extractor - INFO - Extracted 18 race results for 201809020611
2025-08-31 21:56:44,829 - src.scraping.scrapers.race_scraper - INFO - Successfully extracted race data: 201809020611 (18 horses)
2025-08-31 21:56:44,830 - src.scraping.storage.postgresql_storage - INFO - Race Is: Race(race_id='201809020611', race_date=datetime.date(2018, 4, 8), track_name='阪神', race_number=11, race_name='第78回桜花賞(GI)', distance=1600, track_type='芝', total_horses=18, grade='G1', track_direction='右', weather='晴', track_condition='良', start_time=datetime.time(15, 40), winning_time='1:33.1', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)


現在処理中のレース: Race(race_id='201809020611', race_date=datetime.date(2018, 4, 8), track_name='阪神', race_number=11, race_name='第78回桜花賞(GI)', distance=1600, track_type='芝', total_horses=18, grade='G1', track_direction='右', weather='晴', track_condition='良', start_time=datetime.time(15, 40), winning_time='1:33.1', pace=None, prize_1st=None, race_class=None, race_conditions=None, created_at=None, updated_at=None)
